In [ ]:
import polars as pl
from pathlib import Path
import numpy as np
from datetime import datetime, timedelta
import random

In [ ]:
demo_folder = Path("icu_demo_data/aki/mimic_demo/")
dyn = pl.read_parquet(demo_folder / "dyn.parquet")

In [ ]:
dyn.schema

In [ ]:
def generate_data(
    stay_ids: list[int],
    max_rows: int,
    start_time: datetime,
    timestep: timedelta,
    columns: list[str],
    missingness: dict[str, float],
) -> pl.DataFrame:
    """
    Generate a Polars DataFrame with the specified number of rows and schema for multiple stay IDs.

    Parameters:
    - stay_ids: The list of stay IDs to use.
    - max_rows: The maximum number of rows to generate.
    - start_time: The starting time for the time column.
    - timestep: The time interval between rows.
    - columns: A list of column names to generate data for.
    - missingness: A dictionary where keys are column names and values are the percentage of missing values.

    Returns:
    - A Polars DataFrame with the generated data.
    """
    # Distribute rows randomly among stay IDs
    rows_per_stay_id = np.random.multinomial(max_rows, [1 / len(stay_ids)] * len(stay_ids))

    all_data = []

    for stay_id, num_rows in zip(stay_ids, rows_per_stay_id):
        # Generate the time column
        time_column = [start_time + i * timestep for i in range(num_rows)]

        # Initialize the data dictionary with the time and stay_id columns
        data = {"stay_id": [stay_id] * num_rows, "time": time_column}

        # Generate random data for other columns
        for column in columns:
            col_data = np.random.rand(num_rows)
            # Introduce missingness
            if column in missingness:
                missing_count = int(num_rows * missingness[column])
                missing_indices = np.random.choice(num_rows, missing_count, replace=False)
                col_data[missing_indices] = np.nan
            data[column] = col_data

        # Create the DataFrame for the current stay_id
        df = pl.DataFrame(data)
        all_data.append(df)

    # Concatenate all DataFrames
    final_df = pl.concat(all_data)

    return final_df


# Example usage
max_rows = 100000
stay_ids = random.sample(range(1, 10000), int(max_rows / 24))
start_time = datetime(2024, 1, 1)
timestep = timedelta(hours=1)

# Define the list of column names
columns = [
    "alb",
    "alp",
    "alt",
    "ast",
    "be",
    "bicar",
    "bili",
    "bili_dir",
    "bnd",
    "bun",
    "ca",
    "cai",
    "ck",
    "ckmb",
    "cl",
    "crea",
    "crp",
    "dbp",
    "fgn",
    "fio2",
    "glu",
    "hgb",
    "hr",
    "inr_pt",
    "k",
    "lact",
    "lymph",
    "map",
    "mch",
    "mchc",
    "mcv",
    "methb",
    "mg",
    "na",
    "neut",
    "o2sat",
    "pco2",
    "ph",
    "phos",
    "plt",
    "po2",
    "ptt",
    "resp",
    "sbp",
    "temp",
    "tnt",
    "urine",
    "wbc",
]

# Define the missingness dictionary
missingness = {item: random.uniform(0, 0.99) for item in columns}

generated_df = generate_data(stay_ids, max_rows, start_time, timestep, columns, missingness)

In [ ]:
generated_df

In [ ]:
missingness